(chap8)= 

# 第八章: 近似贝叶斯计算 

<style>p{text-indent:2em;2}</style>

在本章中，我们讨论近似贝叶斯计算（ABC）。 ABC中的“近似”是指缺乏显式似然，而不是使用数值方法来近似后验，例如马尔可夫链蒙特卡洛或变分推理。 ABC 方法的另一个常见且更明确的名称是似然方法，尽管一些作者标记了这些术语之间的区别，其他人可以互换使用它们。

当我们没有明确的可能性表达式时，ABC 方法可能很有用，但我们有一个能够生成合成数据的参数化*模拟器*。模拟器有一个或多个未知参数，我们想知道哪一组参数生成的合成数据*足够接近*观测数据。在这个程度上，我们将计算这些参数的后验分布。

ABC 方法在生物科学中变得越来越普遍，特别是在系统生物学、流行病学、生态学和群体遗传学等子领域{cite:p}`Sunnaker2013`。但它们也用于其他领域，因为它们提供了一种灵活的方式来解决许多实际问题。

这种多样性也反映在可用于 ABC {cite:p}`Dutta2017、Lintusaari2018、Klinger2018` 的 Python 包中。然而，额外的近似层也带来了一系列困难。主要定义在没有可能性的情况下*足够接近*意味着什么，然后能够实际计算一个近似的后验。
我们将在本章中从一般的角度讨论这些挑战。我们强烈建议有兴趣将 ABC 方法应用于他们自己的问题的读者，用他们自己的领域知识中的示例来补充本章。

(life-beyond-likelihood)= 

## 8.1 超越似然 

根据贝叶斯定理（方程 [eq:posterior_dist](eq:posterior_dist)），要计算后验，我们需要两个基本成分，先验和似然。但是，对于特定问题，我们可能会发现我们无法以封闭形式表达似然性，或者计算它的成本过高。对于我们的贝叶斯热情来说，这似乎是一条死胡同。但只要我们能够以某种方式生成合成数据，情况就不一定如此。这种合成数据生成器通常被称为*模拟器*。从 ABC 方法的角度来看，模拟器是一个黑盒，我们在一侧输入参数值并从另一侧获取模拟数据。然而，我们添加的复杂性是不确定哪些输入足以生成与观测数据相似的合成数据。

所有 ABC 方法共有的基本概念是用 $\delta$ 函数替换似然性，该函数计算距离或更一般地，观察数据 $Y$ 与合成数据 $\hat Y$ 之间的某种形式的差异参数化模拟器 $Sim$。

```{math} 
\hat Y \sim Sim(\theta)
```

```{math} 
p(\theta \mid Y) 
    \underset{\sim}{\propto} 
    \delta(Y, \hat Y \mid \epsilon)\; p(\boldsymbol{\theta})
```

我们的目标是使用函数 $\delta$ 来获得*实际足够好的*近似*真实*可能性：

```{math} 
\lim_{\epsilon \to 0} \delta(Y, \hat Y \mid \epsilon) = p(Y \mid \boldsymbol{\theta})
```

我们引入了一个容差参数 $\epsilon$，因为对于大多数问题 [^1]，生成与观察数据 $Y$ 相等的合成数据集 $\hat Y$ 的机会几乎为零。 $\epsilon$ 的值越大，我们就越能容忍 $Y$ 和 $\hat Y$ 必须有多接近才能将它们视为*足够接近*。一般来说，对于一个给定的问题，较大的 $\epsilon$ 值意味着对后验的更粗略的近似，我们将在后面看到这方面的例子。

在实践中，随着我们增加数据的样本大小（或维度），为距离函数 $\delta$ [^2] 找到足够小的值变得越来越困难。一个简单的解决方案是增加$\epsilon$ 的值，但这意味着增加我们的近似误差。更好的解决方案可能是使用一个或多个统计量 $S$ 并计算数据汇总之间的距离，而不是模拟数据集和真实数据集之间的距离。

```{math} 
\delta\left(S(Y), S(\hat Y) \mid \epsilon\right)
```
我们必须知道，使用统计量会给 ABC 近似带来额外的误差源，除非统计量对于模型参数 $\theta$ 来说是足够的。

不幸的是，这并不总是可能的。然而，不充分的统计量在实践中仍然非常有用，并且它们经常被从业者使用。

在本章中，我们将探讨一些不同的距离和统计量数据，重点关注一些经过验证的方法。但是要知道，ABC 处理了如此多不同类型的模拟数据，在如此多的不同领域中，可能很难一概而论。此外，文献进展非常迅速，因此我们将专注于构建必要的知识、技能和工具，因此随着 ABC 方法的不断发展，您会发现更容易推广到新问题。

::: {admonition} 充分统计量

如果没有从同一样本计算的其他统计量提供有关该样本的任何附加信息，则统计量对于模型参数就足够了。换句话说，该统计数据*足以*汇总您的样本而不会丢失信息。例如，给定来自具有期望值 $\mu$ 和已知有限方差的正态分布的独立值样本，样本均值对于 $\mu$ 来说是一个**充分的统计量**。请注意，均值没有说明色散，因此仅就参数 $\mu$ 而言就足够了。众所周知，对于 iid 数据，具有足够统计量且维度等于 $\theta$ 维度的唯一分布是来自指数族 {cite:p}`Darmois1935、Koopman1936、Pitman1936、Andersen1970`的分布。对于其他分布，充分统计量的维度随着样本量的增加而增加。

:::


(approximating-the-approximated-posterior)= 

## 8.2 估计近似后验 

执行近似贝叶斯计算的最基本方法可能是拒绝采样。我们将使用 {numref}`fig:abc_rejection` 以及对算法的高级逐步描述来描述它，如下所示。

1. 从先验分布中采样 $\theta$ 的值。

2. 将该值传递给模拟器并生成合成数据。

3. 如果合成数据的*距离* $\delta$ 比 $\epsilon$ 更近，则保存建议的 $\theta$，否则拒绝它。

4. 重复直到获得所需数量的样本。

```{figure} figures/ABC_rejection.png
:name: fig:abc_rejection
:width: 4.5in

ABC 拒绝采样器的一个步骤。我们从先验分布（顶部）中采样一组 $\theta$ 值。每个值都被传递给模拟器，模拟器生成合成数据集（虚线分布），我们将合成数据与观测数据（底部的分布）进行比较。在这个例子中，只有 $\theta_1$ 能够生成一个与观察数据足够接近的合成数据集，因此 $\theta_0$ 和 $\theta_2$ 被拒绝。请注意，如果我们使用统计量信息，而不是整个数据集，我们应该在第 2 步之后和第 3 步之前计算合成数据和观察数据的统计量信息。

``` 

ABC 拒绝采样器的主要缺点是，如果先验分布与后验分布相差太大，我们将花费大部分时间提出将被拒绝的值。更好的想法是从更接近实际后验的分布中提出。

通常，我们对后验的了解不够，无法手动执行此操作，但我们可以使用顺序蒙特卡洛 (SMC) 方法来实现。这是一种通用的采样器方法，就像我们在书中使用的 MCMC 方法一样。 SMC 可以适应执行 ABC，因此称为 SMC-ABC。如果你想了解更多关于 SMC 方法的细节，你可以阅读 Section {ref}`inference_methods`，但要理解这一章，你只需要知道 SMC 通过增加辅助参数 $\beta$ 的值来进行$s$ 连续阶段 $\{\beta_0=0 < \beta_1 < ... < \beta_s=1\}$。这样做的方式是，我们从先验（$\beta = 0$）开始采样，直到我们到达后验（$\beta = 1$）。因此，我们可以将$\beta$ 视为*逐渐开启可能性*的参数。 $\beta$ 的中间值由 SMC 自动计算。关于先验的数据信息越多和/或后验的几何形状越复杂，SMC 将采取的中间步骤越多。

{numref}`fig:smc_tempering` 显示了一个假设的中间分布序列，从浅灰色的先验到蓝色的后验。

```{figure} figures/smc_tempering.png
:name: fig:smc_tempering
:width: 8.00in

SMC 采样器探索的回火后验的假设序列，从浅灰色的先验 ($\beta = 0$) 到蓝色的实际后验 ($\beta = 1$)。开始时较低的 $\beta$ 值有助于采样器不会卡在单个最大值中。

``` 

(fitting-a-gaussian-the-abc-way)= 

## 8.3 用 ABC 拟合一个高斯 

让我们用一个简单的例子来热身，从均值为 0 和标准差为 1 的高斯分布数据中估计均值和标准差。对于这个问题，我们可以拟合模型：

```{math} 
:label: eq:Gauss_model

\begin{split}
    \boldsymbol{\mu} \sim &\; \mathcal{N}(0, 1) \\
    \boldsymbol{\sigma} \sim &\; \mathcal{HN}(1) \\
    \boldsymbol{s} \sim &\; \mathcal{N}(\boldsymbol{\mu}, \boldsymbol{\sigma})

\end{split}
```

在 PyMC3 中编写此模型的直接方法显示在代码块 [gauss_nuts](gauss_nuts) 中。

```{code-block} ipython3 
:name: gauss_nuts
:caption: gauss_nuts 

with pm.Model() as gauss:     
  μ = pm.Normal("μ", mu=0, sigma=1)     
  σ = pm.HalfNormal("σ", sigma=1)     
  s = pm.Normal("s", μ, σ, observed=data)     
  trace_g = pm.sample()
```

使用 SMC-ABC 的等效模型显示在代码块 [gauss_abc](gauss_abc) 中。

```{code-block} ipython3 
:name: gauss_abc
:caption: gauss_abc 

with pm.Model() as gauss:     
  μ = pm.Normal("μ", mu=0, sigma=1)     
  σ = pm.HalfNormal("σ", sigma=1)     
  s = pm.Simulator("s", normal_simulator, params=[μ, σ],
                        distance="gaussian",                      sum_stat="sort",                        epsilon=1,                      observed=data)
  trace_g = pm.sample_smc(kernel="ABC")
```

我们可以看到代码块 [gauss_nuts](gauss_nuts) 和代码块 [gauss_abc](gauss_abc) 之间有两个重要的区别：

- 使用`pm.Simulator` *分布*

- 使用 `pm.sample_smc(kernel="ABC")` 代替 `pm.sample()`。

通过使用`pm.Simulator`，我们告诉PyMC3，我们不会对可能性使用封闭形式的表达式，而是定义一个伪可能性。我们需要传递一个生成合成数据的 Python 函数，在这个例子中是函数“normal_simulator”，以及它的参数。代码块 [normal_simulator](normal_simulator) 显示了此函数的定义，样本大小为 1000，参数未知 $\mu$ 和 $\sigma$。

```{code-block} ipython3
:name: normal_simulator
:caption: normal_simulator

def normal_simulator(μ, σ):
    return np.random.normal(μ, σ, 1000)
```

我们可能还需要向 `pm.Simulator` 传递其他可选参数，包括距离函数 `distance`、统计量信息 `sum_stat` 和容差参数 $\epsilon$ `epsilon` 的值。稍后我们将详细讨论这些论点。我们还将观测数据以常规可能性传递给模拟器分布。

通过使用 `pm.sample_smc(kernel="ABC")`[^3] 我们告诉 PyMC3 在模型中寻找 `pm.Simulator` 并使用它来定义伪似然，其余的采样过程与 SMC 算法中描述的相同。当 `pm.Simulator` 存在时，其他采样器将无法运行。

最后一个成分是 `normal_simulator` 函数。原则上我们可以使用任何我们想要的 Python 函数，实际上我们甚至可以包装非 Python 代码，例如 Fortran 或 C 代码。这就是 ABC 方法的灵活性所在。在这个例子中，我们的模拟器只是一个 NumPy 随机生成器函数的包装器。

与其他采样器一样，建议我们运行多个链，以便我们可以诊断采样器是否无法正常工作，PyMC3 将尝试自动执行此操作。 {numref}`fig:trace_g` 显示了使用两条链运行代码块 [gauss_abc](gauss_abc) 的结果。

我们可以看到我们能够恢复真实的参数，并且采样器没有显示任何明显的采样问题。

```{figure} figures/trace_g.png
:name: fig:trace_g
:width: 8.00in

正如预期的那样，$\mu\approx 0$ 和 $\sigma\approx 1$，两条链都同意 KDE 和秩图反映的后验。
请注意，这两条链中的每一条都是通过运行 2000 个并行 SMC 链/粒子获得的，如 SMC 算法中所述。

``` 

(choosing-the-distance-function-epsilon-and-the-summary-statistics)= 

## 8.4 选择距离函数、 $\epsilon$ 和统计量 

定义有用的距离、统计量和 $\epsilon$ 取决于问题。这意味着我们应该在获得好的结果之前进行一些试验和错误，尤其是在遇到新问题时。像往常一样，首先考虑好的选择有助于减少选择的数量。但是我们也应该接受运行实验，因为它们总是有助于更好地理解问题并对这些超参数做出更明智的决定。在接下来的部分中，我们将讨论一些通用指南。

(choosing-the-distance)= 

### Choosing the Distance 

我们使用默认距离函数 `distance="gaussian"` 运行代码块 [gauss_abc](gauss_abc)，其定义为：

```{math} 
:label: eq:euclidean_abc

\sum_i - \frac{||X_{oi} - X_{si}||^2}{2 \epsilon_i^2}

```

其中 $X_{o}$ 是观测数据，$X_{s}$ 是模拟数据，$\epsilon$ 是其缩放参数。我们称 {eq}`eq:euclidean_abc` *Gaussian* 因为它是对数尺度的高斯核[^4]。我们使用对数尺度来计算伪似然，就像我们使用实际似然（和先验）[^5]。 $||X_{oi} - X_{si}||^2$ 是欧几里得距离（也称为 L2 范数），因此我们也可以将方程 {eq}`eq:euclidean_abc` 描述为加权欧几里得距离。这是文学中非常流行的选择。其他流行的选项是 L1 范数（绝对差的总和），在 PyMC3 中称为拉普拉斯距离、L$\infty$ 范数（差的最大绝对值）或马氏距离：$\sqrt{(xo - xs )^{T}\Sigma(xo - xs)}$，其中 $\Sigma$ 是协方差矩阵。

诸如高斯、拉普拉斯等距离可以应用于整个数据，或者正如我们已经提到的，可以应用于统计量。还专门引入了一些距离函数，以避免统计量的需要，并且仍然提供良好的结果 {cite:p}`Perez2008,jiang2018,Bernton_2019`。我们将讨论其中的两个，Wasserstein 距离和 KL 散度。

在代码块 [gauss_abc](gauss_abc) 我们使用 `sum_stat="sort"` [^6]，这告诉 PyMC3 在计算方程 {eq}`eq:euclidean_abc` 之前对数据进行排序。这样做相当于计算 1D 2-Wasserstein 距离，如果我们这样做但使用 L1 范数，我们将得到 1D 1-Wasserstein 距离。可以为大于 1 {cite:p}`Bernton_2019` 的维度定义 Wasserstein 距离。

在计算距离之前对数据进行排序使得分布之间的比较更加公平。想象一下，我们有两个完全相等的样本，但是出于运气，一个是从低到高排序的，另一个是从高到低排序的。在这种情况下，如果我们应用像方程 {eq}`eq:euclidean_abc` 这样的度量，我们会得出结论，两个样本非常不同，即使它们是相同的样本。但是如果我们先排序，我们会得出结论它们是相同的。这是一个非常极端的场景，但它有助于阐明数据排序背后的直觉。还有一件事，如果我们对数据进行排序，我们假设我们只关心数据的分布而不关心数据的顺序，否则排序会破坏数据中的结构。例如，时间序列可能会发生这种情况，请参见第 [6] 章（第 4 章）。

为避免定义统计量而引入的另一个距离是使用 KL 散度（参见第 {ref}`DKL` 部分）。使用以下表达式 {cite:p}`Perez2008,jiang2018` 来近似 KL 散度：

```{math} 
:label: eq:kl_abc

\frac{d}{n}  \sum \left(- \frac{\log(\frac{\nu_d}{\rho_d})}{\epsilon} \right) + \log\left(\frac{n}{n-1}\right)

```

其中 $d$ 是数据集的维度（变量或特征的数量），$n$ 是观测数据点的数量。 $\nu_d$ 包含观测数据到模拟数据的 1-最近邻距离，$\rho_d$ 包含观测数据到自身的 2-最近邻距离（请注意，如果您将数据集与其自身进行比较，则 1-最近邻距离将永远为零）。由于该方法涉及最近邻搜索的 2n 次操作，因此通常使用 k-d 树 {cite:p}`Bentley1975` 来实现。

(choosing-epsilon)= 

### 选择 $\epsilon$ 

在许多 ABC 方法中，$\epsilon$ 参数用作硬阈值，生成距离大于 $\epsilon$ 的样本的 $\theta$ 值将被拒绝。此外，$\epsilon$ 可以是用户必须设置的递减值列表，或者算法自适应找到 [^7]。

在 PyMC3 中，$\epsilon$ 是距离函数的尺度，就像在方程 {eq}`eq:euclidean_abc` 中一样，所以它不能作为硬阈值。我们可以根据需要设置$\epsilon$。我们可以选择一个标量值（相当于将所有 $i$ 的 $\epsilon_i$ 设置为相等）。这在评估数据上的距离而不是使用统计量数据时很有用。在这种情况下，合理的猜测可能是数据的经验标准偏差。如果我们改为使用统计量，那么我们可以将 $\epsilon$ 设置为值列表。这通常是必要的，因为每个统计量数据可能具有不同的规模。如果尺度差异太大，那么每个统计量的贡献将是不均匀的，甚至可能出现单个统计量主导计算距离的情况。在这些情况下，$\epsilon$ 的一个流行选择是在先验预测分布下的 $i^{\text{th}}$ 统计量的经验标准偏差，或中值绝对偏差，因为这对异常值更稳健.使用先验预测分布的一个问题是它可能比后验预测分布更广泛。因此，为了找到一个有用的 $\epsilon$ 值，我们可能希望将前面提到的这些有根据的猜测作为上限，然后从这些值中尝试一些较低的值。然后我们可以根据计算成本、所需的精度/误差水平和采样器的效率等几个因素来选择 $\epsilon$ 的最终值。一般来说，$\epsilon$ 的值越低，近似值就越好。

{numref}`fig:trace_g_many_eps` 显示了 $\mu$ 和 $\sigma$ 的几个 $\epsilon$ 值以及“NUTS”采样器的森林图（使用正常似然而不是模拟器）。

```{figure} figures/trace_g_many_eps.png
:name: fig:trace_g_many_eps
:width: 8.00in

$\mu$ 和 $\sigma$ 的森林图，使用 NUTS 或 ABC 获得，$\epsilon$、1、5 和 10 的值递增。

``` 

减小$\epsilon$ 的值是有限制的，过低的值会使采样器非常低效，表明我们的目标是一个没有太大意义的准确度水平。

{numref}`fig:trace_g_eps_too_low` 显示了当来自代码块 [gauss_abc](gauss_abc) 的模型以 `epsilon=0.1` 的值进行采样时，SMC 采样器如何无法收敛。正如我们所看到的，采样器非常失败。

```{figure} figures/trace_g_eps_too_low.png
:name: fig:trace_g_eps_too_low
:width: 8.00in

模型`trace_g_001`的KDE和等级图，收敛失败可能表明$\epsilon=0.1$的值对于这个问题来说太低了。

``` 

为了帮助我们为 $\epsilon$ 确定一个好的值，我们可以从我们一直用于非 ABC 方法的模型批评工具中获得帮助，例如贝叶斯 p 值和后验预测检查，如图 {numref}`fig:bpv_g_many_eps_00`、{numref}`fig:bpv_g_many_eps_01` 和 {numref}`fig:ppc_g_many_eps`。

{numref}`fig:bpv_g_many_eps_00` 包括值 $\epsilon=0.1$。我们在这里这样做是为了展示校准不佳的模型的样子。但在实践中，如果我们获得像 {numref}`fig:trace_g_eps_too_low` 中的排名图，我们应该停止分析计算的后验并重新检查模型定义。

此外，对于 ABC 方法，我们还应该检查超参数 $\epsilon$ 的值、我们选择的统计量或距离函数。

```{figure} figures/bpv_g_many_eps_00.png
:name: fig:bpv_g_many_eps_00
:width: 8.00in

$\epsilon$ 值递增的边际贝叶斯 p 值分布。对于一个校准良好的模型，我们应该期待一个均匀分布。我们可以看到 $\epsilon=0.1$ 的校准很糟糕，这并不奇怪，因为 $\epsilon$ 的值也是如此。对于 $\epsilon$ 的所有其他值，分布看起来更加均匀，并且均匀性水平随着 $\epsilon$ 的增加而降低。
`se` 值是预期的均匀分布和计算的 KDE 之间的（缩放的）平方差。

``` 

```{figure} figures/bpv_g_many_eps_01.png
:name: fig:bpv_g_many_eps_01
:width: 8.00in

增加 epsilon 值的贝叶斯 p 值。蓝色曲线是观察到的分布，灰色曲线是预期分布。对于一个校准良好的模型，我们应该期望分布集中在 0.5 左右。我们可以看到 $\epsilon=0.1$ 的校准很糟糕，这并不奇怪，因为这个 $\epsilon$ 的值太低了。
我们可以看到 $\epsilon=1$ 提供了最好的结果。

``` 

```{figure} figures/ppc_g_many_eps.png
:name: fig:ppc_g_many_eps
:width: 8.00in

后验预测检查 $\epsilon$ 的增加值。蓝色曲线是观察到的分布，灰色曲线是预期分布。令人惊讶的是，从 $\epsilon=0.1$ 我们得到了似乎是一个很好的调整，即使我们知道来自该后验的样本不可信，这是一个非常简单的例子，我们完全靠运气得到了正确的答案。这是 *a too good to be true fit* 的一个例子。这些是最糟糕的！如果我们只考虑具有看起来合理的后验样本的模型（即不是 $\epsilon=0.1$ ），我们可以看到 $\epsilon=1$ 提供了最好的结果。

``` 

(choosing-summary-statistics)= 

### 选择汇总的统计量

摘要统计的选择可以说比距离函数的选择更困难，并且会产生更大的影响。出于这个原因，许多研究都集中在这个主题上，从使用不需要统计量的距离 {cite:p}`Jiang2018, Bernton_2019` 到选择统计量的策略 {cite:p}`Sisson2018`。

一个好的统计量提供了低维度和信息量之间的平衡。当我们没有足够的统计量数据时，很容易通过添加大量统计量数据来进行过度补偿。直觉是信息越多越好。然而，增加统计量的数量实际上会降低近似后验 {cite:p}`Sisson2018` 的质量。对此的一种解释是，我们从计算数据上的距离转移到计算摘要上的距离以减少维度，通过增加我们正在违背该目的的摘要统计数据的数量。

在一些领域，如群体遗传学，ABC 方法非常普遍，人们开发了大量有用的统计量数据 {cite:p}`Beaumont2002, Beaumont2010, Pudlo2015`。一般来说，查看您正在研究的应用领域的文献以了解其他人在做什么是一个好主意，因为他们已经尝试并测试了许多替代方案的机会很高。

如有疑问，我们可以遵循上一节中的相同建议来评估模型拟合，即排名图、贝叶斯 p 值、后验预测检查等，并在必要时尝试替代方案（参见图 {numref}`fig:trace_g_eps_too_low`， {numref}`fig:bpv_g_many_eps_00`、{numref}`fig:bpv_g_many_eps_01` 和 {numref}`fig:ppc_g_many_eps`）。

(g-and-k-distribution)= 

## 8.5 `g-and-k` 分布 

一氧化碳 (CO) 是一种无色、无味的气体，大量吸入可能有害，甚至致命。当某物燃烧时会产生这种气体，尤其是在氧气含量低的情况下。世界上许多城市通常会监测一氧化碳和其他气体，如二氧化氮 (NO2)，以评估空气污染程度和空气质量。在城市中，二氧化碳的主要来源是汽车，以及其他通过燃烧化石燃料工作的车辆或机械。 {numref}`fig:co_ppm_bsas` 显示了 2010 年至 2018 年布宜诺斯艾利斯市一个站点测量的每日 CO 水平的直方图。

正如我们所见，数据似乎略微偏右。

此外，数据显示了一些具有非常高值的观察结果。

底部面板省略了 3 到 30 之间的 8 个观察值。

```{figure} figures/co_ppm_bsas.png
:name: fig:co_ppm_bsas
:width: 8.00in

CO 水平的直方图。顶部面板显示整个数据，底部面板忽略大于 3 的值。

``` 

为了拟合这些数据，我们将引入单变量 g-and-k 分布。这是一个 4 参数分布，能够描述具有高偏度和/或峰度的数据 {cite:p}`Tukey1977, Rayner2002`。它的密度函数在封闭形式中不可用，并且 g-and-k 分布通过其分位数函数定义，即累积分布函数的倒数：


```{math} 
:label: eq:g_and_k
a + b \ \left(1 + c \ \text{tanh}\left[\frac{gz(x)}{2}\right]\right) \left(1+z(x)^2\right)^k z(x)

```

其中 $z$ 是标准正态累积分布函数和 $x \in (0,1)$ 的倒数。

参数$a$、$b$、$g$和$k$分别是位置、尺度、偏度和峰度参数。如果 $g$ 和 $k$ 为 0，我们恢复具有均值 $a$ 和标准差 $b$ 的高斯分布。

$g > 0$ 给出正（右）偏度，$g < 0$ 给出负（左）偏度。参数 $k \geqslant 0$ 给出的尾巴比正常的长，而 $k < 0$ 的尾巴比正常的短。 $a$ 和 $g$ 可以取任何实际值。通常将 $b$ 限制为正数并且 $k \geqslant -0.5$ 或有时 $k \geqslant 0$ 即尾部与高斯分布中的尾部一样重或更重。此外，通常修复 $c=0.8$。有了所有这些限制，我们可以保证得到一个严格递增的分位数函数 {cite:p}`Rayner2002`，这是定义明确的连续分布函数的标志。

代码块 [gk_quantile](gk_quantile) 定义了 g-and-k 分位数分布。我们省略了 cdf 和 pdf 的计算，因为它们涉及更多一点，但最重要的是因为我们不会将它们用于我们的示例 [^8]。虽然 g-and-k 分布的概率密度函数可以数值计算 {cite:p}`Rayner2002, prangle2017`，但使用反演方法从 g-and-k 模型进行模拟更直接和快速{cite:p }`Drovandi2011, prangle2017`。为了实现反演方法，我们对 $x \sim \mathcal{U}(0, 1)$ 进行采样并替换为方程 {eq}`eq:g_and_k`。代码块 [gk_quantile](gk_quantile) 展示了如何在 Python 中执行此操作，{numref}`fig:gk_quantile` 展示了 g-and-k 分布的示例。
 

```{code-block} ipython3
:name: gk_quantile
:caption: gk_quantile

class g_and_k_quantile:
    def __init__(self):
        self.quantile_normal = stats.norm(0, 1).ppf

    def ppf(self, x, a, b, g, k):
        z = self.quantile_normal(x)
        return a + b * (1 + 0.8 * np.tanh(g*z/2)) * ((1 + z**2)**k) * z

    def rvs(self, samples, a, b, g, k):
        x = np.random.normal(0, 1, samples)
        return ppf(self, x, a, b, g, k)
```

```{figure} figures/gk_quantile.png
:name: fig:gk_quantile
:width: 8.00in

第一行显示分位数函数，也称为累积分布函数的反函数。你给它一个分位数的值，它会返回代表该分位数的变量的值。

例如，如果您有 $P(X <= x_q) = q$，则将 $q$ 传递给分位数函数并得到 $x_q$。第二行显示（近似的）pdf。

对于此示例，已使用内核密度估计从代码块 [gk_quantile](gk_quantile) 生成的随机样本中计算出 pdf。

``` 

要使用 SMC-ABC 拟合 g-k 分布，我们可以使用高斯距离和 `sum_stat="sort"`，就像我们在高斯示例中所做的那样。

或者，我们也可以考虑为这个问题量身定制的统计量。众所周知，参数 $a$、$b$、$g$ 和 $k$ 分别与位置、比例、偏度和峰度相关联。因此，我们可以根据对这些数量 {cite:p}`Drovandi2011` 的稳健估计来考虑一个统计量：

```{math} 
\begin{split}
sa &= e4 \\
sb &= e6 - e2 \\  
sg &= (e6 + e2 - 2*e4)/sb \\ 
sk &= (e7 - e5 + e3 - e1)/sb \\
\end{split}
```
其中 $e1$ 到 $e7$ 是八分位数，即将样本分成八个子集的分位数。

如果我们注意，我们可以看到 $sa$ 是中位数，$sb$ 是四分位数范围，它们是位置和分散的稳健估计量。即使 $sg$ 和 $sk$ 看起来有点模糊，它们也分别是偏度 {cite:p}`Bowley1920` 和峰态 {cite:p}`Moors1988` 的稳健估计量。让我们更清楚地说明这一点。对于对称分布，$e6-e4$ 和 $e2-e4$ 将具有相同的幅度但符号相反，因此在这种情况下 $sg$ 将为零，对于偏斜分布，$e6-e4$ 将大于 $ e2-e4$ 或反之亦然。

当 $e6$ 和 $e2$ 附近的质量减少时，即当我们将质量从分布的中心部分*移动*到尾部时，$sk$ 的分子中的两项增加。 $sg$ 和 $sk$ 中的分母都充当标准化因子。

考虑到这个想法，我们可以使用 Python 为我们的问题创建一个统计量信息，如以下代码块中指定的那样。

```python
def octo_summary(x):
    e1, e2, e3, e4, e5, e6, e7 = np.quantile(
        x, [.125, .25, .375, .5, .625, .75, .875])
    sa = e4
    sb = e6 - e2
    sg = (e6 + e2 - 2*e4)/sb
    sk = (e7 - e5 + e3 - e1)/sb
    return np.array([sa, sb, sg, sk])
```

现在我们需要定义一个模拟器，我们可以将之前在代码块 [gk_quantile](gk_quantile) 中定义的 `g_and_k_quantile()` 函数中的 `rvs` 方法包装起来。

```python
gk = g_and_k_quantile()
def gk_simulator(a, b, g, k):
    return gk.rvs(len(bsas_co), a, b, g, k)
```

在定义了统计量和模拟器并导入了数据之后，我们可以定义我们的模型。对于这个例子，我们基于所有参数都被限制为正的事实使用弱信息先验。 CO 水平不能取负值，因此 $a$ 为正值，并且 $g$ 也预计为 0 或正值，因为大多数常见水平预计为“低”，一些测量值取较大值。我们也有理由假设参数最有可能低于 1。

```python
with pm.Model() as gkm:
    a = pm.HalfNormal("a", sigma=1)
    b = pm.HalfNormal("b", sigma=1)
    g = pm.HalfNormal("g", sigma=1)
    k = pm.HalfNormal("k", sigma=1)
    
    s = pm.Simulator("s", gk_simulator,
    params=[a, b, g, k],        
                     sum_stat=octo_summary,
                     epsilon=0.1,
                     observed=bsas_co)
    
    trace_gk = pm.sample_smc(kernel="ABC", parallel=True)
```

{numref}`fig:plot_pair` 显示了拟合的 `gkm` 模型的配对图。

```{figure} figures/pair_gk.png
:name: fig:plot_pair
:width: 8.00in

分布略微偏斜，并且具有一定程度的峰度，正如少数 CO 水平所预期的那样，其值比大部分 CO 值大一到两个数量级。我们可以看到 $b$ 和 $k$ （略微）相关。这是可以预期的，随着尾部密度（峰度）的增加，离散度增加，但如果 $k$ 增加，g-and-k 分布可以保持 $b$ 较小。就像 $k$ 是分散的*吸收*部分，类似于我们在学生 t 分布中使用尺度和 $\nu$ 参数观察到的情况。

``` 

(ABC_MA)= 

## 8.6 移动平均的近似 

移动平均 (MA) 模型是建模单变量时间序列的常用方法（参见第 [6] 章（第 4 章））。 MA(q) 模型指定输出变量线性依赖于随机项 $\lambda$ 的当前值和 $q$ 以前的过去值。 $q$ 被称为 MA 模型的阶数。

```{math} 
y_t = \mu + \lambda_t + \theta_1 \lambda_{t-1} + \cdots + \theta_q \lambda_{t-q}
```

其中 $\lambda$ 是高斯白噪声误差项 [^9]。

我们将使用取自 {cite:t}`Marin2012` 的玩具模型。对于这个例子，我们将使用平均值为 0 的 MA(2) 模型（即
$\mu =0$)，因此我们的模型如下所示：


```{math} 
y_t = \lambda_t + \theta_1 \lambda_{t-1} +  \theta_2 \lambda_{t-2}
```

代码块 [ma2_simulator_abc](ma2_simulator_abc) 显示了此模型的 Python 模拟器，在 {numref}`fig:ma2_simulator_abc` 中，我们可以看到来自该模拟器的值 $\theta1 = 0.6、\theta2=0.2$ 的两个实现。

 
```{code-block} ipython3
:name: ma2_simulator_abc
:caption: ma2_simulator_abc

def moving_average_2(θ1, θ2, n_obs=200):
    λ = np.random.normal(0, 1, n_obs+2)
    y = λ[2:] + θ1*λ[1:-1] + θ2*λ[:-2]
    return y
```

```{figure} figures/ma2_simulator_abc.png
:name: fig:ma2_simulator_abc
:width: 8.00in

MA(2) 模型的两种实现，一种具有 $\theta1=0.6，\theta2 =0.2$。
左列是核密度估计，右列是时间序列。

``` 

原则上，我们可以尝试使用我们想要的任何距离函数和/或统计量来拟合 MA(q) 模型。相反，我们可以使用 MA(q) 模型的一些属性作为指导。 MA(q) 模型中经常感兴趣的一个属性是它们的自相关性。理论表明，对于 MA(q) 模型，大于 q 的滞后将为零，因此对于 MA(2)，使用滞后 1 和滞后 2 的自相关函数作为统计量似乎是合理的。此外，只是为了为了避免计算数据的方差，我们将使用自协方差函数而不是自相关函数。

```python
def autocov(x, n=2):
    return np.array([np.mean(x[i:] * x[:-i]) for i in range(1, n+1)])
```

此外，除非我们引入一些限制，否则 MA(q) 模型是不可识别的。对于 MA(1) 模型，我们需要限制 $-1<\theta_1<1$。

对于 MA(2)，我们有 $-2<\theta_1<2$、$\theta_1+\theta_2>-1$ 和 $\theta_1-\theta_2<1$，这意味着我们需要从三角形中采样，如{numref}`fig:ma2_triangle`。

结合自定义统计量信息和可识别的限制，ABC 模型在代码块 [MA2_abc](MA2_abc) 中指定。

```{code-block} ipython3
:name: MA2_abc
:caption: MA2_abc

with pm.Model() as m_ma2:
    θ1 = pm.Uniform("θ1", -2, 2)
    θ2 = pm.Uniform("θ2", -1, 1)
    p1 = pm.Potential("p1", pm.math.switch(θ1+θ2 > -1, 0, -np.inf))
    p2 = pm.Potential("p2", pm.math.switch(θ1-θ2 < 1, 0, -np.inf))

    y = pm.Simulator("y", moving_average_2, 
                     params=[θ1, θ2],
                     sum_stat=autocov,
                     epsilon=0.1,
                     observed=y_obs)

    trace_ma2 = pm.sample_smc(3000, kernel="ABC")
```

`pm.Potential` 是一种将任意项合并到（伪）似然的方法，无需向模型添加新变量。引入限制特别有用，就像在这个例子中一样。在代码块 [MA2_abc](MA2_abc) 中，如果 `pm.math.switch` 中的第一个参数为真，则我们将 0 与可能性相加，否则为 $-\infty$。

```{figure} figures/ma2_trace.png
:name: fig:ma2_trace
:width: 8.00in

MA(2) 模型的 ABC 轨迹图。正如预期的那样，真实参数被恢复，秩图看起来非常平坦。

``` 

```{figure} figures/ma2_triangle.png
:name: fig:ma2_triangle
:width: 8.00in

代码块 [MA2_abc](MA2_abc) 中定义的 MA(2) 模型的 ABC 后验。在中心子图中，联合后验和边缘中 $\theta1$ 和 $\theta2$ 的边际分布。灰色三角形代表先验分布。平均值用黑点表示。

``` 

(model-comparison-in-the-abc-context)= 

## 8.7 ABC 语境下的模型比较 

ABC 方法经常用于模型选择。虽然已经提出了许多方法 {cite:p}`Sisson2018, Beaumont2019`，但这里我们将讨论两种方法；贝叶斯因子，包括与 LOO 的比较和随机森林 {cite:p}`Pudlo2015`。

与参数推断一样，摘要的选择对于模型比较至关重要。如果我们使用它们的预测评估两个或多个模型，如果它们都做出大致相同的预测，我们就不能偏爱一个模型。相同的推理可以应用于具有统计量的 ABC 下的模型选择。如果我们使用均值作为统计量，但模型预测的均值相同，那么此统计量将不足以区分模型。

我们应该花更多的时间来思考是什么让模型与众不同。

(marginal-likelihood-and-loo)= 

### Marginal Likelihood and LOO 

用于执行 ABC 方法的模型比较的一个常见量是边际似然。通常，这种比较采用边际似然比的形式，称为贝叶斯因子。如果贝叶斯因子的值大于 1，则分子中的模型优于分母中的模型，反之亦然。在 {ref}`Bayes_factors` 部分中，我们讨论了有关贝叶斯因子的更多细节，包括它们的注意事项。一个这样的警告是边际似然通常难以计算。幸运的是，SMC 方法和扩展的 SMC-ABC 方法能够将边际似然计算为抽样的副产品。 PyMC3 的 SMC 计算并保存跟踪中的对数边际似然。我们可以通过执行 `trace.report.log_marginal_likelihood` 来访问它的值。由于该值采用对数刻度，因此要计算贝叶斯因子，我们可以这样做：

```python
ml1 = trace_1.report.log_marginal_likelihood
ml2 = trace_2.report.log_marginal_likelihood
np.exp(ml1 - ml2)
```

当使用统计量时，通常不能信任从 ABC 方法计算的边际似然来区分竞争模型 {cite:p}`Robert2011`，除非统计量足以进行模型比较。这令人担忧，因为除了一些正式的示例或特定模型之外，没有通用指南来确保模型 {cite:p}`Robert2011` 的充分性。如果我们使用所有数据，即我们不依赖统计量 [^10]，这不是问题。这类似于我们的讨论（参见第 {ref}`Bayes_factors`），即计算边际似然通常是一个比计算后验要困难得多的问题。即使我们设法找到一个足以计算后验的统计量，也不能保证它对模型比较也有用。

为了更好地理解边际似然在 ABC 方法的背景下如何表现，我们现在将分析一个简短的实验。我们还包括 LOO，因为我们认为 LOO 是比边际可能性和贝叶斯因子更好的整体指标。

我们实验的基本设置是将具有显式似然性的模型的对数边际似然值和使用 LOO 计算的值与使用带有和不带有统计量的模拟器的 ABC 模型的值进行比较。结果显示在代码块 [gauss_nuts](gauss_nuts) 和 [gauss_abc](gauss_abc) 中的图 {numref}`fig:model_comp_normal_0` 模型中。边际（伪）似然的值由 SMC 的乘积和 LOO 的值使用 `az.loo()` 计算得出。请注意，LOO 是在逐点对数似然值上正确定义的，但在 ABC 中，我们只能访问逐点对数-*伪*似然值。

从 {numref}`fig:model_comp_normal_0` 我们可以看到，通常 LOO 和对数边际似然的行为相似。从第一列中我们看到，`model_1` 始终被选为比 `model_0` 更好（这里越高越好）。模型之间的差异（斜率）对于对数边际似然比 LOO 更大，这可以解释为边际似然的计算明确考虑了先验，而 LOO 仅通过后验间接进行（参见第 {ref }`Bayes_factors` 了解详情）。即使 LOO 的值和边际似然因样本而异，它们也会以一致的方式进行。我们可以从连接 `model_0` 和 `model_1` 的线的斜率中看到这一点。虽然线的斜率并不完全相同，但它们非常相似。这是模型选择方法的理想行为。如果我们比较`model_1`和`model_2`，我们可以得出类似的结论。另外考虑到两种模型对于 LOO 基本上无法区分，而边际似然反映了更大的差异。再一次，原因是 LOO 仅从后验计算，而边际似然直接考虑了先验。

```{figure} figures/model_comp_normal_00.png
:name: fig:model_comp_normal_0
:width: 8.00in

模型 `m_0` 与方程 {eq}`eq:Gauss_model` 中描述的模型相似，但具有 $\sigma \sim \mathcal{HN}(0.1)$。 `model_1` 与方程 {eq}`eq:Gauss_model` 相同。 `model_2` 与方程 {eq}`eq:Gauss_model` 相同，但使用 $\sigma \sim \mathcal{HN}(10)$。
第一行对应于对数边际似然的值，第二行对应于使用 LOO 计算的值。顺序蒙特卡洛`SMC`，SMC-ABC 与整个数据集`SMC-ABC`，SMC-ABC 使用平均值作为统计量`SMC-ABC_sm`，最后 SMC-ABC 使用平均值和标准偏差`SMC-ABC_sq` .我们进行了 50 个实验，每个实验的样本量为 50。

``` 

第二列显示了当我们进入 ABC 领域时会发生什么。我们仍然选择 `model_1` 作为更好的选择，但现在 `model_0` 的离散度比来自 `model_1` 或 `model_2` 的离散度要大得多。

此外，我们现在得到了相互交叉的线。综合起来，这两个观察似乎表明我们仍然可以使用 LOO 或对数边际似然来选择最佳模型，但是相对权重的值，例如由 `az.compare()` 或贝叶斯因子计算的值将具有较大的可变性。

第三列显示了当我们使用平均值作为统计量时会发生什么。现在模型 `model_0` 和 `model_1` 看起来差不多，而 `model_2` 看起来是个糟糕的选择。它几乎就像上一栏的镜面图像。这表明当使用带有统计量的 ABC 方法时，对数边际似然和 LOO 可能无法提供合理的答案。

第四列显示了当我们使用除均值之外的标准差作为统计量时会发生什么。我们看到，我们可以定性地恢复将 ABC 与整个数据集（第二列）一起使用时观察到的行为。

::: {admonition} 在伪似然的尺度上

请注意 y 轴上的比例是如何不同的，尤其是跨列。原因有两个，首先，当使用 ABC 时，我们使用按 $\epsilon$ 缩放的核函数来逼近似然性，其次，当使用统计量时，我们正在减小数据的大小。另请注意，如果我们增加平均值或分位数等统计量的样本量，则该大小将保持不变，即，无论我们从 10 次还是 1000 次观察中计算平均值，平均值都是一个数字。

:::

{numref}`fig:model_comp_normal_forest` 可以帮助我们理解我们刚刚从 {numref}`fig:model_comp_normal_0` 讨论的内容。我们建议您自己分析这两个数字。目前，我们将重点关注两个观察结果。首先，在执行“SMC-ABC_sm”时，我们有足够的均值统计信息，但对数据的分散性无话可说，因此参数“a”和“σ”的后验不确定性基本上由先验控制。看看 `model_0` 和 `model_1` 的估计值对于 `μ` 是如何非常相似的，而 `model_2` 的不确定性非常大。其次，关于参数“σ”，“model_0”的不确定性非常小，“model_1”的不确定性应该更大，“model_2”的不确定性大得离谱。综上所述，我们可以看到为什么对数边际似然和 LOO 表明 `model_0` 和 `model_1` 相当，但 `model_2` 却非常不同。基本上，`SMC-ABC_sm` 不能很好地匹配！一旦我们看到，从 SMC-ABC_sm 计算的对数边际似然和 LOO 与我们使用 SMC 或 SMC-ABC 时观察到的相矛盾就不足为奇了。如果我们使用均值和标准差“SMC-ABC_sq”作为统计量，我们可以部分恢复使用整个数据集“SMC-ABC”的行为。

```{figure} figures/model_comp_normal_forest.png
:name: fig:model_comp_normal_forest
:width: 8.00in

模型 `m_0` 与方程 {eq}`eq:Gauss_model` 中描述的模型相似，但具有 $\sigma \sim \mathcal{HN}(0.1)$。 `model_1` 与方程 {eq}`eq:Gauss_model` 相同。 `model_2` 与方程 {eq}`eq:Gauss_model` 相同，但使用 $\sigma \sim \mathcal{HN}(10)$。

第一行包含边际似然值，第二行包含 LOO 值。该列表示计算这些值的不同方法。顺序蒙特卡洛`SMC`，SMC-ABC 与整个数据集`SMC-ABC`，SMC-ABC 使用平均值作为统计量`SMC-ABC_sm`，最后 SMC-ABC 使用平均值和标准偏差`SMC-ABC_sq` .我们进行了 50 个实验，每个实验的样本量为 50。
``` 
图 {numref}`fig:model_comp_pois_geom_0` 和 {numref}`fig:model_comp_pois_geom_forest` 显示了类似的分析，但 `model_0` 是几何模型，而 `model_1` 是 Poisson 模型。数据遵循移位泊松分布 $\mu\sim 1 + \text{Pois}(2.5)$。我们将这些数字的分析留给读者作为练习。

```{figure} figures/model_comp_pois_geom_00.png
:name: fig:model_comp_pois_geom_0
:width: 8.00in

模型 `m_0` 是具有先验 $p \sim \mathcal{U}(0, 1)$ 的几何分布，而 `model_1` 是具有先验 $\mu \sim \mathcal{E}(1)$ 的泊松分布。数据遵循移位泊松分布 $\mu\sim 1 + \text{Pois}(2.5)$。顺序蒙特卡洛`SMC`，SMC-ABC 与整个数据集`SMC-ABC`，SMC-ABC 使用平均值作为统计量`SMC-ABC_sm`，最后 SMC-ABC 使用平均值和标准差`SMC-ABC_sq` .我们进行了 50 个实验，每个实验的样本量为 50。

``` 

```{figure} figures/model_comp_pois_geom_forest.png
:name: fig:model_comp_pois_geom_forest
:width: 8.00in

`model_0` 具有先验 $p \sim \mathcal{U}(0, 1)$ 的几何模型/模拟器 `model_1` 具有先验 $p \sim \text{Expo}(1)$ 的泊松模型/模拟器。第一行包含边际似然的值，第二行包含 LOO 的值。该列表示计算这些值的不同方法。顺序蒙特卡洛`SMC`，SMC-ABC 与整个数据集`SMC-ABC`，SMC-ABC 使用平均值作为统计量`SMC-ABC_sq`，最后 SMC-ABC 使用四分位数`SMC-ABC_sq`。我们进行了 50 个实验，每个实验的样本量为 50。

``` 

在 ABC 文献中，通常使用贝叶斯因子来尝试将相对概率分配给模型。我们知道这在某些领域被认为是有价值的。所以我们想警告那些从业者在 ABC 框架下这种做法的潜在问题，特别是因为使用统计量比不使用要普遍得多。模型比较仍然有用，主要是如果采用更具探索性的方法以及在模型比较之前执行模型批评以改进或丢弃明显错误指定的模型。这是我们在本书中为非 ABC 方法采用的一般方法，因此我们认为将其扩展到 ABC 框架也是很自然的。在本书中，我们也偏爱 LOO 而不是边际可能性，虽然目前缺乏关于 LOO 对 ABC 方法的优缺点的研究，但我们认为 LOO 也可能对 ABC 方法有用。请继续关注未来的消息！

::: {admonition} 模型批评和模型比较

虽然总是会出现一些错误规格，但模型比较可以帮助更好地理解模型及其错误规格。只有在我们证明模型对数据提供了合理的拟合之后，才应该进行模型比较。比较明显不合适的模型没有太大意义。

:::

(model-choice-via-random-forest)= 

### Model Choice via Random Forest 

我们在上一节中讨论的注意事项激发了对 ABC 框架下模型选择新方法的研究。一种这样的替代方法将模型选择问题重新定义为随机森林分类问题 {cite:p}`Pudlo2015` [^11]。随机森林是一种基于许多决策树组合的分类和回归方法，它与第 [7] 章（第 6 章）中的 BART 密切相关。

该方法的主要思想是，可以通过从先验或后验预测分布的模拟构建随机森林分类器来获得最可能的模型。在原始论文中，作者使用了先验预测分布，但提到对于更高级的 ABC 方法，也可以使用其他分布。在这里，我们将使用后验预测分布。对于最多 $m$ 个模型的每个模型，模拟在参考表中排序（参见 {numref}`table:ABC_random_forest_ref_table`）。

其中每一行是来自后验预测分布的样本，每一列是 $n$ 统计量中的一个。我们使用这个参考表来训练分类器，任务是在给定统计量值的情况下正确分类模型。重要的是要注意，用于模型选择的统计量不需要与用于计算后验的相同。事实上，建议包括许多统计量信息。一旦分类器被训练，我们就用我们在参考表中使用的相同的 $n$ 统计量信息来提供它，但这次应用于我们观测数据。分类器预测的模型将是我们最好的模型。

 

```{list-table} Reference table
:name: table:ABC_random_forest_ref_table

* - **Model**
  - $\mathbf{S^{0}}$
  - $\mathbf{S^{1}}$
  - ...
  - $\mathbf{S^{n}}$
* - 0
  -
  -
  - ...
  -
* - 0
  -
  -
  - ...
  -
* - ...
  - ...
  - ...
  - ...
  - ...
* - 1
  -
  -
  - ...
  -
* - 1
  -
  -
  - ...
  -
* - m
  -
  -
  - ...
  -
```

此外，我们还可以计算 *best* 模型相对于其余模型的近似后验概率。再一次，这可以使用随机森林来完成，但这次我们使用回归，将错误分类错误率作为响应变量，将参考表中的统计量作为自变量 {cite:p}`Pudlo2015`。

(model-choice-for-ma-model)= 

### Model Choice for MA Model 

让我们回到移动平均线的例子，这次我们将重点关注以下问题。 MA(1) 或 MA(2) 是更好的选择吗？为了回答这个问题，我们将使用 LOO（基于逐点伪似然值）和随机森林。 MA(1) 模型看起来像这样

```{code-block} ipython3
:name: MA1_abc
:caption: MA1_abc

with pm.Model() as m_ma1:
    θ1 = pm.Uniform("θ1", -1, 1)
    y = pm.Simulator("y", moving_average_1,
                     params=[θ1], sum_stat=autocov, epsilon=0.1, observed=y_obs)
    trace_ma1 = pm.sample_smc(2000, kernel="ABC")
```

为了比较使用 LOO 的 ABC 模型。我们不能直接使用函数 `az.compare`。我们首先需要创建一个带有 `log_likelihood` 组的 `InferenceData` 对象，详见代码块 [idata_pseudo](idata_pseudo) [^12]。此比较的结果总结在 {numref}`table:abc_loo` 中。正如预期的那样，我们可以看到 MA(2) 模型是首选。

```{code-block} ipython3
:name: idata_pseudo
:caption: idata_pseudo

idata_ma1 = az.from_pymc3(trace_ma1)
lpll = {"s": trace_ma2.report.log_pseudolikelihood}
idata_ma1.log_likelihood = az.data.base.dict_to_dataset(lpll)

idata_ma2 = az.from_pymc3(trace_ma2)
lpll = {"s": trace_ma2.report.log_pseudolikelihood}
idata_ma2.log_likelihood = az.data.base.dict_to_dataset(lpll)

az.compare({"m_ma1":idata_ma1, "m_ma2":idata_ma2})
```


```{list-table} Summary ABC-model comparison using LOO
:name: table:abc_loo
* -
  - **rank**
  - **loo**
  - **p_loo**
  - **d_loo**
  - **weight**
  - **se**
  - **dse**
  - **warning**
  - **loo_scale**
* -  model_ma2
  -    0
  -  -2.22
  -   1.52
  -   0.00
  -   1.0
  -   0.08
  -   0.00
  - False
  - log
* -  model_ma1
  -    1
  -  -3.53
  -   2.04
  -   1.31
  -   0.0
  -   1.50
  -   1.43
  - False
  - log
```

要使用随机森林方法，我们可以使用本书随附代码中包含的 `select_model` 函数。为了使这个函数工作，我们需要传递一个带有 PyMC3 模型名称和跟踪的元组列表、一个统计量列表和观测数据。这里作为统计量，我们将使用前六个自相关。我们选择这些特定的统计量有两个原因，第一个表明我们可以使用一组不同于用于拟合数据的统计量，第二个表明我们可以混合有用的统计量（前两个自相关） , 没有非常有用的（其余的）。请记住，理论说对于一个 MA(q) 过程，最多有 q 个自相关。对于复杂的问题，例如来自群体遗传学的问题，使用数百甚至数万个统计量数据并不少见 {cite:p}`Collin2020`。

```python
from functools import partial
select_model([(m_ma1, trace_ma1), (m_ma2, trace_ma2)],
             statistics=[partial(autocov, n=6)],
             n_samples=5000,
             observations=y_obs)
```

`select_model` 返回最佳模型的索引（从 0 开始）和该模型的估计后验概率。对于我们的示例，我们得到模型 0 的概率为 0.68。至少在这个例子中，LOO 和随机森林方法都同意模型选择甚至它们的相对权重，这有点让人放心。

(choosing-priors-for-abc)= 

## 8.8 为 ABC 选择先验 

没有封闭形式的可能性使得获得好的模型变得更加困难，因此 ABC 方法通常比其他近似值更脆弱。因此，我们应该格外小心建模选择，包括先验启发，并且比我们有明确可能性时更彻底地评估模型。这些是我们为近似可能性所付出的成本。

与其他方法相比，使用 ABC 方法进行更仔细的先验启发可能会更有价值。如果我们通过近似可能性来丢失信息，我们可以通过使用更多信息的先验来部分补偿该损失。此外，更好的先验通常会使我们免于浪费计算资源和时间。对于 ABC 拒绝方法，我们使用先验作为抽样分布，这或多或少是显而易见的。但 SMC 方法也是如此，特别是如果模拟器对输入参数敏感。例如，当使用 ABC 推断常微分方程时，某些参数组合可能难以进行数值模拟，从而导致模拟速度极慢。在 SMC 和 SMC-ABC 的加权采样过程中出现了使用模糊先验的另一个问题，因为在对回火后验进行评估时，除了少数先验样本外，几乎所有样本的权重都非常小。这导致 SMC 粒子在几个步骤后变得奇异（因为只选择了少数重量较大的样本）。这种现象称为重量崩溃，这是粒子方法 {cite:p}`bickel2008sharp` 的一个众所周知的问题。

良好的先验可以帮助降低计算成本，从而在一定程度上允许我们在使用 SMC 和 SMC-ABC 时拟合更复杂的模型。除了提供更多信息的先验的一般建议以及我们在本书其他地方已经讨论过的关于先验启发/评估的内容之外，我们没有针对 ABC 方法的进一步建议。

(exercises8)= 

## 8.9 练习 

**8E1.** In your words explain how ABC is approximate? What object or quantity is approximated and how.

**8E2.** In the context of ABC, what is the problem that SMC is trying to solve compared to rejection sampling? 

**8E3.** Write a Python function to compute the Gaussian kernel as in Equation {eq}`eq:euclidean_abc`, but without the summation.

Generate two random samples of size 100 from the same distribution. Use the implemented function to compute the distances between those two random samples. You will get two distributions each of size 100. Show the differences using a KDE plot, the mean and the standard deviation.

**8E4.** What do you expect to the results to be in terms of accuracy and convergence of the sampler if in model `gauss` model from Code Block [gauss_abc](gauss_abc) we would have used `sum_stat="identity"`. Justify.

**8E5.** Refit the `gauss` model from Code Block [gauss_abc](gauss_abc) using `sum_stat="identity"`.

Evaluate the results using: 

1.  Trace Plot 

2.  Rank Plot 

3.  $\hat R$ 

4.  The mean and HDI for the parameters $\mu$ and $\sigma$.

Compare the results with those from the example in the book (i.e. using `sum_stat="sort"`).

**8E6.** Refit the `gauss` model from Code Block [gauss_abc](gauss_abc) using quintiles as summary statistics.

1. How the results compare with the example in the book? 

2.  Try other values for `epsilon`. Is 1 a good choice? 

**8E7.** Use the `g_and_k_quantile` class to generate a sample (n=500) from a g-and-k distribution with parameters a=0,b=1,g=0.4,k=0. Then use the `gkm` model to fit it using 3 different values of $\epsilon$ (0.05, 0.1, 0.5). Which value of $\epsilon$ do you think is the best for this problem? Use diagnostics tools to help you answer this question.

**8E8.** Use the sample from the previous exercise and the `gkm` model. Fit the using the summary statistics `octo_summary`, the `octile-vector` (i.e. the quantiles 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875) and `sum_stat="sorted"`. Compare the results with the known parameter values, which option provides higher accuracy and lower uncertainty? 

**8M9.** In the GitHub repository you will find a dataset of the distribution of citations of scientific papers. Use SMC-ABC to fit a g-and-k distribution to this dataset. Perform all the necessary steps to find a suitable value for `"epsilon"` and ensuring the model converge and results provides a suitable fit.

**8M10.** The Lotka-Volterra is well-know biological model describing how the number of individuals of two species change when there is a predator-prey interaction {cite:p}`Otto2007`. Basically, as the population of prey increase there is more food for the predator which leads to an increase in the predator population. But a large number of predators produce a decline in the number of pray which in turn produce a decline in the predator as food becomes scarce. Under certain conditions this leads to an stable cyclic pattern for both populations.

In the GitHub repository you will find a Lotka-Volterra simulator with unknown parameters and the data set `Lotka-Volterra_00`. Assume the unknown parameters are positive. Use a SMC-ABC model to find the posterior distribution of the parameters.

 **8H11.** Following with the Lotka-Volterra example. The dataset `Lotka-Volterra_01` includes data for a predator prey with the twist that at some point a disease suddenly decimate the prey population. Expand the model to allow for a "switchpoint", i.e. a point that marks two different predator-prey dynamics (and hence two different set of parameters).

 **8H12.** This exercise is based in the sock problem formulated by Rasmus Bååth. The problem goes like this. We get 11 socks out of the laundry and to our surprise we find that they are all unique, that is we can not pair them. What is the total number of socks that we laundry? Let assume that the laundry contains both paired and unpaired socks, we do not have more than two socks of the same kind. That is we either have 1 or 2 socks of each kind.

 Assume the number of socks follows a $\text{NB}(30, 4.5)$. And that the proportion of unpaired socks follows a $\text{Beta}(15, 2)$ 

Generate a simulator suitable for this problem and create a SMC-ABC model to compute the posterior distribution of the number of socks, the proportion of unpaired socks, and the number of pairs.

---

[^1]: It can work for discrete variables, especially if they take only a     few possible values.

[^2]: This is another manifestation of the curse of dimensionality. See     Section {ref}`high_dimensions` for a full explanation.

[^3]: The default SMC `kernel` is `"metropolis"`. See {ref}`inference_methods` for details.

[^4]: Is similar to the Gaussian distribution but without the     normalization term $\frac{1}{\sigma\sqrt{2\pi}}$.

[^5]: This is something PyMC3 does, other packages could be different 

[^6]: Even when PyMC3 uses `sum_stat="sort"` as summary statistic,     sorting is not a true summary as we are still using the whole data 

[^7]: In a similar fashion as the $\beta$ parameters in the description     of the SMC/SMC-ABC algorithm explained before 

[^8]: In Prangle {cite:p}`prangle2017` you will find a description of an R     package with a lot of functions to work with g-and-k distributions.

[^9]: In the literature is common to use $\varepsilon$ to denote these     terms, but we want to avoid confusion with the $\epsilon$ parameter     in the SMC-ABC sampler 

[^10]: Good moment to remember that `sum_stat="sort"` is not actually a     summary statistic as we are using the entire dataset 

[^11]: Other classifiers could have been chosen, but the authors decided     to use a random forest.

 [^12]: In future versions of PyMC `pm.sample_smc` will return and     InferenceData object with the proper groups.